<a href="https://colab.research.google.com/github/sruthics12/DCVST/blob/master/ILLUMU_Assignment_C_S_Sruthi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Using web scrapping tools, gather data from Reddit (https://www.reddit.com/r/Anxiety/) posted in the last 3 months
2. Create a database of only 500 sentences
3. take only 200 sentences from the database to annotate each sentence with 0-1 indicators. 0 means not anxious, and 1 means anxious. We understand that you are not a psychologist, so take your best guess while annotating.
4. Build a model that learns on the annotated portion of the database  
5. Come up with prediction of anxiety of the 300 other sentences from your algorithm

In [ ]:
#installing library for reddit scrapping
!pip install praw
import praw
import pandas as pd

     |████████████████████████████████| 167 kB 5.4 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 


In [ ]:
import time

In [ ]:
# Acessing the reddit api
reddit = praw.Reddit(client_id="",#my client id
                     client_secret="",  #your client secret
                     user_agent="", #user agent name
                     username = "",     # your reddit username
                     password = "")     # your reddit password

In [ ]:
# Creating a dictionary to store the data scraped
post_dict = {
    "title" : [],
    "score" : [],
    "id" : [],
    "url" : [],
    "comms_num": [],
    "created" : [],
    "body" : []}
for submission in reddit.subreddit('Anxiety').hot(limit=100):
  post_dict["title"].append(submission.title)
  post_dict["score"].append(submission.score)
  post_dict["id"].append(submission.id)
  post_dict["url"].append(submission.url)
  post_dict["comms_num"].append(submission.num_comments)
  post_dict["created"].append(submission.created)
  post_dict["body"].append(submission.selftext)

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
#converting dictionary to a pandas dataframe for data preparation
post_data = pd.DataFrame(post_dict)
post_data.head(10)

,title,score,id,url,comms_num,created,body
0,Let your light shine!,14,pt717k,https://www.reddit.com/r/Anxiety/comments/pt71...,5,1.632316e+09,Congrats /r/anxiety we've all made it to Wedne...
1,Monthly Check-In Thread,72,pbnr64,https://www.reddit.com/r/Anxiety/comments/pbnr...,235,1.629936e+09,Hello everyone! Welcome to the r/Anxiety month...
2,Am I a disgusting person?,204,pul1ru,https://www.reddit.com/r/Anxiety/comments/pul1...,51,1.632494e+09,I am 16 and a male and have OCD. I’ll constant...
3,I'm a grown man and I'm afraid to get my 2nd P...,234,pujh57,https://www.reddit.com/r/Anxiety/comments/pujh...,185,1.632489e+09,The 1st shot I was a mess beforehand. Even the...
4,God I fucking hate work,44,pusy1t,https://www.reddit.com/r/Anxiety/comments/pusy...,7,1.632519e+09,Work just amplifies my social anxiety to the p...
5,Took action against my harasser.,21,puvs31,https://www.reddit.com/r/Anxiety/comments/puvs...,3,1.632529e+09,I’ve finally gathered up enough courage to tak...
6,"""get out of your comfort zone""",64,puosbq,https://www.reddit.com/r/Anxiety/comments/puos...,8,1.632505e+09,I don't have a god damn comfort zone. I have p...
7,Does anyone else get bursts of energy where th...,17,puuxof,https://www.reddit.com/r/Anxiety/comments/puux...,4,1.632526e+09,"My anxiety has gotten better over the years, b..."
8,Anyone else feel anxiety over every little thing?,8,puwoks,https://www.reddit.com/r/Anxiety/comments/puwo...,5,1.632533e+09,Anyone else have crippling anxiety over everyt...
9,Does anyone feel like their anxiety is made wo...,171,puf04i,https://www.reddit.com/r/Anxiety/comments/puf0...,22,1.632470e+09,


In [ ]:
created_t=  post_data['created'].apply(lambda x: time.ctime(x))

In [ ]:
#the top 100 posts used here were posted on 24 September, within the required time duration of 3 months
created_t.tail(5)

95    Fri Sep 24 19:22:13 2021
96    Fri Sep 24 23:07:48 2021
97    Fri Sep 24 22:56:10 2021
98    Fri Sep 24 22:42:43 2021
99    Fri Sep 24 22:39:26 2021
Name: created, dtype: object

In [ ]:
post_data['body'].tail(10)

90    After some research, I have come to the conclu...
91    Hello, I am 22 y/o and just started lexapro. I...
92    I have had almost constant anxiety for the pas...
93    Yeah it's a low dose, 1x per day as needed wit...
94                                                     
95    Not sure really how to word this but I have a ...
96    Every single time im close to a test and it se...
97    Does anyone ever want to claw or scratch their...
98    Does anyone else here have ocd and is scared o...
99    Does anyone get to the point where they’re too...
Name: body, dtype: object

In [ ]:
a = list(post_data['body'])

In [ ]:
# converting list into string and then joining it with space
b = ' '.join(str(e) for e in a)
# now this is just a long pararaph of text, further we will convert it into lowercase because man and Man is interpreted differently
b=b.rstrip().lower()
print(b)

congrats /r/anxiety we've all made it to wednesday!

this is a weekly thread that serves as a place for us to shine some light on what is going well in our lives. it is easy for mental health subreddits to focus primarily on the negative, after all we're not here because everything is going wonderfully.

but, once in a while you get shown the light in the strangest of places if you look at it right. so whether you made it out of bed today, spoke up in a meeting or class, rode the bus, or just stepped outside please share with us what went well for you in the past week.

any accomplishment, victory, positive thought or action no matter how big or small is welcome here! hello everyone! welcome to the r/anxiety monthly check-in thread. we hope for this to serve as casual community chat for anyone who wants to get or stay involved without having to make a full post. you can also use this as an easy way to give us feedback on what you like and don't like about the subreddit.

# checking in


In [ ]:
#we split at every fullstop to get sentences
sentences =b.split('.')
len(sentences)

847

In [ ]:
df = pd.DataFrame (sentences, columns = ['text'])

In [ ]:
#all special charecters are removed making use of the regular expressions library
import re
def only_alnum(text):
  result= re.sub(r"[^a-zA-Z0-9]+", ' ', text)
  return result

In [ ]:
text=df['text'].apply(lambda x : only_alnum(x))

In [ ]:
text.head(10)

0    congrats r anxiety we ve all made it to wednes...
1     it is easy for mental health subreddits to fo...
2     but once in a while you get shown the light i...
3     so whether you made it out of bed today spoke...
4     any accomplishment victory positive thought o...
5     we hope for this to serve as casual community...
6     you can also use this as an easy way to give ...
7     checking in let us know what s on your mind t...
8     thanks and stay safe the r anxiety mod team i...
9     i ll constantly be anxious over whether my ac...
Name: text, dtype: object

In [ ]:
#recording the length of the sentences
j=text.apply(lambda x: len(x.split(' ')))

In [ ]:
j.mean()

20.272727272727273

In [ ]:
data=pd.concat([text,j], axis=1)
data.columns=['text','no_of_words']

In [ ]:
data.head()

,text,no_of_words
0,congrats r anxiety we ve all made it to wednes...,34
1,it is easy for mental health subreddits to fo...,25
2,but once in a while you get shown the light i...,22
3,so whether you made it out of bed today spoke...,37
4,any accomplishment victory positive thought o...,28


In [ ]:
# Filter all rows for which the no.of words is greater than or equal to 3
#because we need at least 3 words to have a sentence that conveys some meaning on its own
#also removing sentences with more than 25 words because in that case there can be multiple themes and may be difficult to classify
data_f = data[(data['no_of_words'] >= 3) & (data['no_of_words']<25)]
  
# Print the new dataframe
print(data_f.head(15))

                                                 text  no_of_words
2    but once in a while you get shown the light i...           22
8    thanks and stay safe the r anxiety mod team i...           19
11   it s as if through giving into an urge i m fr...           23
12   my cousin came over today and i was having a ...           23
16                       even the xanax barely helped            6
17                                       but i did it            5
18         only has a sore arm and woke up sweaty lol           11
19   but i hear the 2nd shot is more likely to rea...           15
20                       my dad s 2nd he felt nothing            8
21        my teenage siblings felt achey and fatigued            8
22                            hope i don t get it bad            8
23   i m going to drug myself again and go get the...           15
26   work just amplifies my social anxiety to the ...           18
27                                  i hate small talk         

In [ ]:
dataset1=data_f.iloc[:500,:]

In [ ]:
dataset1.shape

(500, 2)

In [ ]:
dataset2= dataset1.reset_index()
dataset=dataset2.drop(['index'], axis=1)

In [ ]:
dataset.head()

,text,no_of_words
0,but once in a while you get shown the light i...,22
1,thanks and stay safe the r anxiety mod team i...,19
2,it s as if through giving into an urge i m fr...,23
3,my cousin came over today and i was having a ...,23
4,even the xanax barely helped,6


In [ ]:
#splitting into testing and training dataset
train=dataset.iloc[:200,:]
test=dataset.iloc[200:500,:]

In [ ]:
test.shape

(300, 2)

In [ ]:
train.shape

(200, 2)

In [ ]:
#exporting the dataset for future use and annotation
dataset.to_csv('dataset.csv')
train.to_csv('train.csv')
test.to_csv('test.csv')

In [ ]:
#extracting labels from the annotated file
train_wl=pd.read_csv('train_a.csv')
train_wl.head()

,Unnamed: 0,text,no_of_words,label
0,0,but once in a while you get shown the light in...,22,0
1,1,thanks and stay safe the r anxiety mod team i ...,19,0
2,2,it s as if through giving into an urge i m fre...,23,0
3,3,my cousin came over today and i was having a t...,23,1
4,4,even the xanax barely helped,6,1


In [ ]:
y_train=train_wl['label']

In [ ]:
#splitting / tokenizing the dataset
dataset= dataset['text'].apply(lambda x: x.split())

In [ ]:
dataset.head()

0    [but, once, in, a, while, you, get, shown, the...
1    [thanks, and, stay, safe, the, r, anxiety, mod...
2    [it, s, as, if, through, giving, into, an, urg...
3    [my, cousin, came, over, today, and, i, was, h...
4                   [even, the, xanax, barely, helped]
Name: text, dtype: object

In [ ]:
pip install nltk

In [ ]:
#removing stop words, We imported a list of the most frequently used words ‘i’, ‘me’, ‘my’, ‘myself’, ‘we’, ‘you’, ‘he’, ‘his’, for example. 
#We usually want to remove these because they have low predictive power.
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'bo

In [ ]:
def remove_stopwords(text):
  words=[w for w in text if w not in stopwords.words('english')]
  return words

In [ ]:
dataset= dataset.apply(lambda x : remove_stopwords(x))

In [ ]:
dataset.head()

0    [get, shown, light, strangest, places, look, r...
1    [thanks, stay, safe, r, anxiety, mod, team, 16...
2    [giving, urge, freeing, anxiety, wrong, may, f...
3    [cousin, came, today, ton, intrusive, sexual, ...
4                        [even, xanax, barely, helped]
Name: text, dtype: object

In [ ]:
#lemmatizing the dataset
lemmatizer = nltk.WordNetLemmatizer()
def word_lemmatizer(text):
  lem_text=[lemmatizer.lemmatize(i) for i in text]
  return lem_text

In [ ]:
nltk.download('wordnet')
dataset=dataset.apply(lambda x : word_lemmatizer(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#transforming individual words to their roots, these words may not be found in the dictionary or even make sense
stemmer= nltk.PorterStemmer()
def word_stemmer(text):
  stem_text=" ".join([stemmer.stem(i) for i in text])
  return stem_text

In [ ]:
dataset=dataset.apply(lambda x : word_stemmer(x))

In [ ]:
dataset.head()

0           get shown light strangest place look right
1       thank stay safe r anxieti mod team 16 male ocd
2                 give urg free anxieti wrong may feel
3    cousin came today ton intrus sexual thought ac...
4                                 even xanax bare help
Name: text, dtype: object

In [ ]:
dataset.shape

(500,)

In [ ]:
#creating a pipeline that first vectorizes the words and then trains classification models on it
# we will be trying the Naive Bayes and Support Vector Machine for classification of the vectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
model1 = make_pipeline(TfidfVectorizer(),MultinomialNB())

In [ ]:
train=dataset.iloc[:200]

In [ ]:
model1.fit(train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [ ]:
test=dataset.iloc[200:500]
y_pred= model1.predict(test)

In [ ]:
test_a=pd.read_csv("test_a.csv")
test_a.head()

,Unnamed: 0,text,no_of_words,label
0,200,while a journal for myself worked as a tool to...,21,0
1,201,i am getting my first vaccine today and i am b...,13,1
2,202,my ocd is going absolutely mad,7,1
3,203,i am nervous for an allergic reaction even tho...,19,1
4,204,the anxiety is making me have a bad headache a...,16,1


In [ ]:
y_test=test_a['label']

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
acc= accuracy_score(y_test, y_pred)
print(f'accuracy of the naive bayes model is %.2f'% (acc*100)+'%')

accuracy of the naive bayes model is 63.67%


In [ ]:
creport = classification_report(y_test, y_pred)
print("Classification report:\n",creport)

Classification report:
               precision    recall  f1-score   support

           0       0.69      0.77      0.73       191
           1       0.50      0.40      0.45       109

    accuracy                           0.64       300
   macro avg       0.60      0.59      0.59       300
weighted avg       0.62      0.64      0.63       300



In [ ]:
#print confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm1=pd.DataFrame(cm, columns=['True','False'],index=['True','False'])
cm1

,True,False
True,147,44
False,65,44


In [ ]:
from sklearn.svm import SVC
model2 = make_pipeline(TfidfVectorizer(),SVC())

In [ ]:
model2.fit(train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape=

In [ ]:
y_pred_svc= model2.predict(test)

In [ ]:
acc= accuracy_score(y_test, y_pred_svc)
print(f'accuracy of the svc model is %.2f'% (acc*100)+'%')

accuracy of the svc model is 65.00%


In [ ]:
creport = classification_report(y_test, y_pred_svc)
print("Classification report:\n",creport)

Classification report:
               precision    recall  f1-score   support

           0       0.67      0.91      0.77       191
           1       0.55      0.20      0.30       109

    accuracy                           0.65       300
   macro avg       0.61      0.55      0.53       300
weighted avg       0.62      0.65      0.60       300



In [ ]:
#confusion matrix for svc
cm = confusion_matrix(y_test, y_pred_svc)
cm1=pd.DataFrame(cm, columns=['True','False'],index=['True','False'])
cm1

,True,False
True,173,18
False,87,22


Naive Bayes shows better Precision and Recall as well as offers a similar accuracy score when compared with the SVC model. The SVC model seems to classify a large portion of the data as negetive. In this use case Naive Bayes seems like a better approach because higher Recall is crucial.

In [ ]:
# Trying an ensemble model for better results
from sklearn.ensemble import RandomForestClassifier
# Instantiating the model with 1000 decision trees
# Train the model on training data
model3 = make_pipeline(TfidfVectorizer(),RandomForestClassifier(n_estimators = 1000, random_state = 42))

In [ ]:
model3.fit(train, y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [ ]:
y_pred_rf= model3.predict(test)

In [ ]:
acc= accuracy_score(y_test, y_pred_rf)
print(f'accuracy of the rf model is %.2f'% (acc*100)+'%')

accuracy of the svc model is 66.33%


In [ ]:
creport = classification_report(y_test, y_pred_rf)
print("Classification report:\n",creport)

Classification report:
               precision    recall  f1-score   support

           0       0.68      0.88      0.77       191
           1       0.58      0.28      0.37       109

    accuracy                           0.66       300
   macro avg       0.63      0.58      0.57       300
weighted avg       0.64      0.66      0.63       300



In [ ]:
#confusion matrix for random forest
cm = confusion_matrix(y_test, y_pred_rf)
cm1=pd.DataFrame(cm, columns=['True','False'],index=['True','False'])
cm1

,True,False
True,169,22
False,79,30


In [ ]:
y_train.value_counts()

0    114
1     86
Name: label, dtype: int64

In [ ]:
y_test.value_counts()

0    191
1    109
Name: label, dtype: int64

# **Including 2-grams for vectorizing and measuring performance with different models**

In [ ]:
import pandas as pd
df=pd.read_csv('dataset.csv')
df.head()

,Unnamed: 0,text,no_of_words
0,0,but once in a while you get shown the light i...,22
1,1,thanks and stay safe the r anxiety mod team i...,19
2,2,it s as if through giving into an urge i m fr...,23
3,3,my cousin came over today and i was having a ...,23
4,4,even the xanax barely helped,6


In [ ]:
df1=df['text'].apply(lambda x : x.split())

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(text):
  words=[w for w in text if w not in stopwords.words('english')]
  return words
df2=df1.apply(lambda x : remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#transforming individual words to their roots, these words may not be found in the dictionary or even make sense
stemmer= nltk.PorterStemmer()
def word_stemmer(text):
  stem_text=" ".join([stemmer.stem(i) for i in text])
  return stem_text

In [ ]:
a=df2.apply(lambda x : word_stemmer(x))

In [ ]:
#a=df2.apply(lambda x : " ".join(x))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
# using default tokenizer in TfidfVectorizer
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
features = tfidf.fit_transform(a)
x=pd.DataFrame(features.todense(),columns=tfidf.get_feature_names())

In [ ]:
x_train=x.iloc[0:200,:]
x_test=x.iloc[200:,:]

In [ ]:
train=pd.read_csv('train_a.csv')
test=pd.read_csv('test_a.csv')
y_train=train['label']
y_test=test['label']

In [ ]:
from sklearn.linear_model import LogisticRegression
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)
y_pred_lr = logisticRegr.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
acc= accuracy_score(y_test, y_pred_lr)
print(f'accuracy of the logistic model is %.2f'% (acc*100)+'%')

accuracy of the logistic model is 65.33%


In [ ]:
from sklearn.svm import SVC
svc= SVC()
svc.fit(x_train,y_train)
y_pred_svc = svc.predict(x_test)

In [ ]:
acc= accuracy_score(y_test, y_pred_svc)
print(f'accuracy of the svc is %.2f'% (acc*100)+'%')

accuracy of the svc is 66.33%


1.33% jump in performance of svc model when we include 2 gram tockens as well